## Бинаризация изображений

In [1]:
import numpy as np
import pandas as pd
#import random
from PIL import Image, ImageDraw
from skimage import io
import matplotlib.pyplot as plt

import cv2

from skimage.filters import (threshold_niblack,
                             threshold_sauvola)

import glob

from sklearn.metrics import f1_score, precision_score, recall_score

from threading import Thread
from datetime import datetime

%matplotlib inline

In [2]:
def paths(original_path, GT_path):
    original_paths = sorted(glob.glob(original_path))
    GT_paths = sorted(glob.glob(GT_path))
    return original_paths, GT_paths

In [3]:
def compare(gray_matrix_t, gray_matrix_p, width, height):

    pixels_array_true = ((np.reshape(gray_matrix_t, width * height)+1) % 2).astype(int)
    pixels_array_pred = ((np.reshape(gray_matrix_p, width * height)+1) % 2).astype(int)
    
    #print(pixels_array_true)
    #print(pixels_array_pred)
    
    precision = precision_score(pixels_array_true, pixels_array_pred)
    recall = recall_score(pixels_array_true, pixels_array_pred)
    f1_metrics = f1_score(pixels_array_true, pixels_array_pred)
    return precision, recall, f1_metrics
    

In [4]:
def load_image(image_path):
    img = Image.open(image_path) #Открываем изображение. 
    #print(image.shape)
    #image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = img.convert('RGB')
    draw = ImageDraw.Draw(image) #Создаем инструмент для рисования. 
    width = image.size[0] #Определяем ширину. 
    height = image.size[1] #Определяем высоту.
    pixels = image.load() #Выгружаем значения пикселей.
    gray_matrix = io.imread(image_path, as_grey=True) # матрица с уровнями серого всех пикселей
    #print(gray_matrix.shape)
    #print(width, height)
    return image, draw, width, height, pixels, gray_matrix

In [5]:
def gray_matrix_to_sorted_series(gray_matrix, width, height):
    pixels_array = np.reshape(gray_matrix, width*height)
    pixels_array_sorted = sorted(pixels_array)
    pixels_series_sorted = pd.Series(pixels_array_sorted)
    return pixels_series_sorted

In [6]:
def plot_pixels_hist(pixels_series_sorted, figsize = (15, 10), width =0.01):
    pixels_series_sorted.hist(bins = 75, figsize = figsize, width = width);

In [7]:
def otsu_algorithm_find_threshold(pixels_series_sorted, width, height):
    pixels_counts_series = pixels_series_sorted.value_counts()
    pixels_counts_series_sorted = pixels_counts_series.sort_index(ascending = True)
    gray_levels = np.array(pixels_counts_series_sorted.index)
    numbers_of_pixels = np.array(pixels_counts_series_sorted.values)
    
    N_0 = width*height
    n = gray_levels.size
    
    w = np.zeros(n-1)
    for k in range(1,n):
        w[k - 1] = np.sum(numbers_of_pixels[:k]) / N_0
        
    mu = np.zeros(n-1)
    for j in range(1, n):
            mu[j - 1] = np.sum((numbers_of_pixels[:j]/N_0)*gray_levels[:j])
          
    var = (((mu[n-2]*w) - mu)**2)/(w*(1 - w))
        
    k_star = var.argmax()
    
    threshold = gray_levels[k_star]
    
    return threshold

In [8]:
def otsu_modified_find_threshold(pixels_series_sorted, width, height):
    pixels_counts_series = pixels_series_sorted.value_counts()
    pixels_counts_series_sorted = pixels_counts_series.sort_index(ascending = True)
    gray_levels = np.array(pixels_counts_series_sorted.index)
    numbers_of_pixels = np.array(pixels_counts_series_sorted.values)
    
    N_0 = width*height
    n = gray_levels.size
   
    
    w = np.zeros(n-1)
    for k in range(1,n):
        w[k - 1] = np.sum(numbers_of_pixels[:k]) / N_0
        
    w1 = 1 - w
    
        
    mu = np.zeros(n-1)
    for j in range(1, n):
            mu[j - 1] = np.sum((numbers_of_pixels[:j]/N_0)*gray_levels[:j])

    mu0 = mu/w
    mu1 = (mu[n-2] - mu)/w1
        
    var0 = np.zeros(n-1)
    var1 = np.zeros(n-1)
    for i in range(1, n):
        var0[i - 1] = np.sum(((gray_levels[:i] -mu0[i-1]) ** 2) * (numbers_of_pixels[:i]/N_0) / w[i-1])
        var1[i - 1] = np.sum(((gray_levels[i:] -mu1[i-1]) ** 2) * (numbers_of_pixels[i:]/N_0) / w1[i-1])
        
    var_w = w*var0 + w1*var1
    
    Q = w*np.log(w) + w1*np.log(w1) - np.log(var_w)
        
    k_star = Q.argmax()
    
    threshold = gray_levels[k_star]
    
    return threshold

In [9]:
def binarize_image(draw, pixels, gray_level_threshold, gray_matrix, width, height):
    for i in range(width):
        for j in range(height):
            red = pixels[i, j][0]
            green = pixels[i, j][1]
            blue = pixels[i, j][2]
            if (gray_matrix[j, i] > gray_level_threshold):
                red, green, blue = 255, 255, 255
            else:
                red, green, blue = 0, 0, 0
            draw.point((i, j), (red, green, blue))
    

In [10]:
def draw_local(draw, pixels, width, height, binary_matrix):
     for i in range(width):
        for j in range(height):
            red = pixels[i, j][0]
            green = pixels[i, j][1]
            blue = pixels[i, j][2]
            if (binary_matrix[j, i] == 1):
                red, green, blue = 255, 255, 255
            else:
                red, green, blue = 0, 0, 0
            draw.point((i, j), (red, green, blue))

In [11]:
def niblack_threshold(gray_matrix, window_size, k, const):
    thresh_niblack = threshold_niblack(gray_matrix, window_size=window_size, k = k)
    binary_niblack = gray_matrix > (thresh_niblack - const)
    return binary_niblack

In [12]:
def savuola_threshold(gray_matrix, window_size, k):
    thresh_sauvola = threshold_sauvola(gray_matrix, window_size=window_size, k = k)
    binary_sauvola = gray_matrix > thresh_sauvola
    return binary_savuola

## Подбор параметров для Ниблэка

In [44]:
ks = [-0.3, -0.4, -0.5]
ws = [3, 5, 7]
consts = [0.03, 0.05, 0.07]
counter1 = 63
#nswers = ['ans0.jpg', 'ans1.jpg', 'ans2.jpg', 'ans3.jpg' ]

In [17]:
search_parametr_results = pd.DataFrame(columns = ["k", "w", "c", "average_f1", "std_f1"])

In [589]:
startTime = datetime.now() 
for k in ks:
    for w in ws:
        for const in consts:
            s = []
            for i in range(0, len(doc_paths), 1):
                image, draw, width, height, pixels, gray_matrix = load_image(doc_paths[i])
                binary_niblack = niblack_threshold(gray_matrix, w, k, const)
                gray_matrix_t = io.imread(doc_GT_paths[i], as_grey=True)
                precision, recall, f1_metrics = compare(gray_matrix_t, binary_niblack, width, height)
                s.append(f1_metrics)
        search_parametr_results.loc[counter1] = [k, w, const, np.mean(s), np.std(s)]
        counter1 += 1   
endTime = datetime.now() 
print ("Время выполнения: ", endTime - startTime)

/home/verochka/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until
/home/verochka/anaconda3/lib/python3.6/site-packages/skimage/filters/thresholding.py:777: RuntimeWarning: invalid value encountered in sqrt
  s = np.sqrt(g2 - m * m)


Время выполнения:  0:01:17.640812


In [45]:
def tread_sum_counter(w, k, const, doc_paths, a, b, j):
    for i in range(a, b, 1):
        image, draw, width, height, pixels, gray_matrix = load_image(doc_paths[i])
        binary_niblack = niblack_threshold(gray_matrix, w, k, const)
        gray_matrix_t = io.imread(doc_GT_paths[i], as_grey=True)
        
        precision, recall, f1_metrics = compare(gray_matrix_t, binary_niblack, width, height)
        s[j].append(f1_metrics)
    

In [864]:
startTime = datetime.now() 
# for k in ks:
#     for w in ws:
#         for const in consts:
#             s = [[], []]  #np.zeros(4)
#             #counter = np.zeros(4)
#             thread0 = Thread(target=tread_sum_counter, args=(w, k, const, doc_paths, 0, 7, 0))
#             thread1 = Thread(target=tread_sum_counter, args=(w, k, const, doc_paths, 7, 15, 1))
#         #thread2 = Thread(target=tread_sum_counter, args=(w, k, original_paths, 80, 120, 2))
#         #thread3 = Thread(target=tread_sum_counter, args=(w, k, original_paths, 120, 150, 3))
#             startTime = datetime.now() 
#             thread0.start()
#             thread1.start()
#         #thread2.start()
#         #thread3.start()
#             thread0.join()
#             thread1.join()
#         #thread2.join()
#         #thread3.join()
#             s = s[0] + s[1]
#             search_parametr_results.loc[counter1] = [k, w, const, np.mean(s), np.std(s)]
#             counter1 += 1  
endTime = datetime.now() 
print ("Время выполнения: ", endTime - startTime)

Время выполнения:  0:00:00.000084


In [22]:
search_parametr_results

,k,w,c,average_f1,std_f1
0,0.1,3.0,0.03,0.796506,0.064841
1,0.1,3.0,0.05,0.700112,0.107190
2,0.1,3.0,0.07,0.555221,0.166605
3,0.1,5.0,0.03,0.876548,0.042161
4,0.1,5.0,0.05,0.875441,0.038380
5,0.1,5.0,0.07,0.838093,0.055202
6,0.1,7.0,0.03,0.887093,0.033628
7,0.1,7.0,0.05,0.913149,0.018208
8,0.1,7.0,0.07,0.899484,0.029740
9,0.2,3.0,0.03,0.745230,0.077836


## Эксперименты

In [856]:
def make_ans(image_path, binary_function):
    image, draw, width, height, pixels, gray_matrix = load_image(image_path)
    pixels_series_sorted = gray_matrix_to_sorted_series(gray_matrix, width, height)
    gray_level_threshold = binary_function(pixels_series_sorted, width, height)
    gray_matrix_p = gray_matrix > gray_level_threshold
    return gray_matrix_p, width, height

In [857]:
def make_ans_local(image_path, binary_function):
    image, draw, width, height, pixels, gray_matrix = load_image(original_paths[i])
    binary_matrix = binary_function(gray_matrix, 27, -0.4, 0.11)
    return binary_matrix, width, height

In [858]:
original_path = '/home/verochka/image_binarization/originals/*.png'
GT_path = '/home/verochka/image_binarization/GT/*.png'
original_paths, GT_paths = paths(original_path, GT_path)

In [859]:
doc_path = '/home/verochka/image_binarization/Im/*.png'
doc_GT_path = '/home/verochka/image_binarization/Im_GT/*.png'
doc_paths, doc_GT_paths = paths(doc_path, doc_GT_path)

In [860]:
original_paths, GT_paths = doc_paths, doc_GT_paths

In [861]:
experiment_results =pd.DataFrame(columns = ["Image_name", "Otsu", "Otsu_modified", "Niblack"])
a = [0, 0, 0]

In [862]:
ans_N = "N7%d.jpg"
ans_O = "O7%d.jpg"
ans_OM = "OM7%d.jpg"

In [863]:
for i in range(len(original_paths)):
    image, draw, width, height, pixels, gray_matrix = load_image(original_paths[i])
    gray_matrix_t = io.imread(GT_paths[i], as_grey=True)
    
    gray_matrix_p, width, height = make_ans(original_paths[i], otsu_algorithm_find_threshold)
    precision, recall, f1_metrics = compare(gray_matrix_t, gray_matrix_p, width, height)
    a[0] = f1_metrics
    draw_local(draw, pixels, width, height, gray_matrix_p)
    ans1 = (ans_O % i)
    image.save(ans1, "JPEG")
    
    gray_matrix_p, width, height = make_ans(original_paths[i], otsu_modified_find_threshold)
    precision, recall, f1_metrics = compare(gray_matrix_t, gray_matrix_p, width, height)
    a[1] = f1_metrics
    draw_local(draw, pixels, width, height, gray_matrix_p)
    ans1 = (ans_OM % i)
    image.save(ans1, "JPEG")
    
    binary_matrix, width, height = make_ans_local(original_paths[i], niblack_threshold)
    precision, recall, f1_metrics = compare(gray_matrix_t, binary_matrix, width, height)
    a[2] = f1_metrics
    draw_local(draw, pixels, width, height, binary_matrix)
    ans1 = (ans_N % i)
    image.save(ans1, "JPEG")
    
    experiment_results.loc[i] = [original_paths[i], a[0], a[1], a[2]]

In [43]:
experiment_results

,Otsu,Otsu_modified,Niblack
0,0.733201,0.767509,0.919118
1,0.457566,0.518061,0.931728
2,0.995137,0.988602,0.932428
3,0.784232,0.837737,0.915985
4,0.990843,0.992104,0.941940
5,0.989272,0.989930,0.933819
6,0.986795,0.987062,0.930125
7,0.843767,0.850837,0.882251
8,0.889002,0.901132,0.919323
9,0.929814,0.936206,0.947385
